In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#plt.style.available
plt.style.use('ggplot')

# Multi layer perceptron

In this practical we fit our first deep neural network. We are starting with the simplest architecture - a multi-layer perceptron (MLP). 

## The task

We have a set of spectra from Muon spectroscopy experiemnts, we would like to train a NN to be able to detect from a given spectrum wheter or not a certain element is present in the sample. In this case we are going to try to train a network to detect if there is `Ag` in a sample.

## The data

There is a dataset conveniently provided with spectra and labels saying wheter or not there is `Ag` in the data (`ag-muon-data.pkl`). It is in a `pickle` file, load it into a `pandas` dataframe and take a quick look.

In [ ]:
df = pd.read_pickle('ag-muon-data.pkl')

In [ ]:
df.head()

Notice that the third column of the dataframe contains either 1 or zero, conveniently the data has already been one-hot-encoded for you. 1 - there is `Ag` in the sample, 0 - there is no `Ag` in the sample. The fifth column of the dataframe contains the spectroscopic signal from the experiment. 

The sample number `23993` is pure `Ag` plot it out to see the kind of data we are looking at.

*Note* To access this data in `pandas` use `sample = df.iloc[23993][3]`

Use `plt.plot(sample)` to look at the data.

We can see some clear peaks associated with `Ag` - however, to get a feel for the complexity of picking out signals with `Ag` in multinary samples try plotting sample number `26`, which is an `Ag/Si` binary.

## The approach

We are going to try and train a NN to classify spectra into those that contain `Ag` and those that do not.

Get `X` - your input data as the fifth column (labelled `3` in the dataframe) of the dataset and `y` your objective as the third column (labelled `1` in the dataframe). You can use the pandas function `to_list` to do this. Convert the resulting lists to `numpy` arrays for feeding to the NN.

```
X = np.array(df[3].to_list())
y = np.array(df[1].to_list())
```

## Try out a network

We can see that the data is given as a series of 300 points, each corresponding to an intensity. We can think about building a MLP that has 300 nodes in the input layer and then a single node in the output layer. The output should be 1 if there is `Ag` in the sample and 0 if there is no `Ag` in the sample.

We will start by using `keras` `Sequential` model building - this allows you to simply add new layers to a model using the `model.add(<layer details>)` command. The model is instantiated with
```
model = Sequential()
```
Now we want to add layers to the netowrk - as we are building a MLP, all of our layers are fully connected or [`Dense`](https://keras.io/layers/core/) layers. The arguments that are passed to the dense layer are
* Number of nodes
* Number of inputs to the layer - this is only necessary for the first layer and must match the shape of the data
* Activation function - this is the non-linear function used to calculate the activation of the layer 
* There are many other properties you can set, but these are all we need for now

**Note** we are importing from `tensorflow.keras`, rather than `keras` this is to ensure consistency with models built in pure `tensorflow`.

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
# define the keras model
model = Sequential()
model.add(Dense(12, input_dim=300, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

We can now look at the properties of the model with `model.summary()`

Now you need to [compile](https://keras.io/models/model/) the model - this involves setting a `loss` to train against, an `optimizer` and optionally some `metrics` to evaluate during the training. For the `loss`, since we are fitting to a range between 0 and 1 `binary_crossentropy` is the obvious choice. For `optimizer` we use `adam` and we will have just one metric `accuracy` - note that metrics must be passed as a list. See the [`keras` documentation](https://keras.io/models/model/)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

When the model is compied we can now start to fit it using the data. Again details of the `fit` function of `keras` models can be found in the [`keras` documentation](https://keras.io/models/model/). 

We need to specify the X and y data, we will run for 60 epochs and we will use a batch size of 32. We should also specify a validation split of 20% of the data using `validation_split=0.2` Run the model for `20` epochs with a `batch_size` of 64.

In [ ]:
model.fit(X, y, epochs=20, batch_size=64, validation_split=0.2)

## Class imbalance

Notice how the accuracy of the model very quickly achieves a certain value and then stops improving. This value could be well above 50%, which suggests that that model is actually learning something. However we may have fallen into a trap here. Take a look at the distribution of the target data, using `plt.hist(y)`. Also pay special attention to the validation data (the final 20% of the data `y[21000:]`).

You might notice something about the class numbers that closely resembls the accuracy achieved by the model. The data with no `Ag` accounts for around 95% of the data. So if the model simply learns to guess no `Ag` it can achieve 95% accuracy without learning anything useful. To avoid this we will rebalabace the classes.

With class imbalance there are a number  of strategies that you can take to address the problem.

* Up-sample the minority class
* Down-sample the majority class
* Change your performance metric

In our case the best available option is to down-sample the majority class, so that `Ag` and no`Ag` have equal numbers in the datasets.

You can split the dataframe into the `1` and `0` entries using something like `df[df[1]==0]`.

Then use [`sklearn.utils.resample`](https://scikit-learn.org/stable/modules/generated/sklearn.utils.resample.html) to resample the majority dataframe.

Then concatenate and shuffle the datasets - `pd.concat` followed by `pd.sample`

A good online guide is available [here](https://elitedatascience.com/imbalanced-classes).

In [ ]:
from sklearn.utils import resample

num_ag = np.count_nonzero(y==0)
num_no_ag = np.count_nonzero(y==1)

df_majority = df[df[1]==0]
df_minority = df[df[1]==1]

# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=num_no_ag,     # to match minority class
                                 random_state=123) # reproducible results

df_downsampled = pd.concat([df_majority_downsampled, df_minority])
df_new = df_downsampled.sample(frac=1)

X = np.array(df_new[3].to_list())
y = np.array(df_new[1].to_list())

## Retrain the network 

Rerun the `model.fit()` command from above using the new data.

**Note** In some instances the model now stops learning at around 50% - pretty disappointing. So what else could be holding us back? In some instances the model will actually train and achieve above 50% in this step - the explanation is below.

## Optimizer hyperparameters

Recall from the lecture that we talked about hyperparameters in optimizers. In this case we have used the `Adam` optimizer with a default learning rate. 

#### "The learning rate is perhaps the most important hyperparameter. If you have time to tune only one hyperparameter, tune the learning rate." - Goodfellow, Deep Learning

One thing that can happen if our learning rate is too high initially is that the optimizer can continually jump over optimal solutions and not actually find its way to a good answer. Whether or not the optimizer does find it's way to a minmum is down to a combination of the loss function space, the starting parameters and the optimizer settings. Since the initial parameters of the network are randomised there are some cases, as noted above where, the network will optimise at the standard learning rate. However, sometimes it will just keep skipping around and not learning. 

To overcome this problem we can lower the learning rate of the optimizer for an initial optimisation. Once the model has started to train and the loss has started to decrease we can dial the learning rate back up to speed up the training.

On the other hand, lower the learning rate too much and training will take an eternity.

### Grid search for learning rate

For this part of the exercise we will do a serach over possible learning rates. Typically we would look over  range of 0.1 to 10$^{-5}$ on a $\log$ scale.

To edit the optimizer settings, we import the optimizer `Adam` from `tensorflow.keras.optimizers` and set the learning rate

```
ad = Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy', optimizer=ad, metrics=['accuracy'])
```

Construct a loop to go over the learning rate range described above. For each training cycle we will extract the best validation accuracy value.

You can save the history of a training cycle as a dictionary simply by `history = model.fit(<params>)`, here the metrics and losses are saved to a dictionary `history.history`. Run the model for 40 epochs at each learning rate.

Save this dictionary for each learning rate and then compare the results. Look at maximum value for `accuracy` from each of the learning rates.

In [ ]:
from tensorflow.keras.optimizers import Adam
lr_histories = []
for lr in [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]:
    model = Sequential()
    model.add(Dense(12, input_dim=300, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    ad = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=ad, metrics=['accuracy'])
    history = model.fit(X, y, epochs=40, batch_size=64, validation_split=0.2)
    lr_histories.append(history.history)

In [ ]:
maxs = [max(h['accuracy']) for h in lr_histories]
plt.plot(maxs)

### Now train the model at the identified learning rate for 100 epochs

Then plot the training history of this run - again taken from the `history.history` dictionary.

In [ ]:
model = Sequential()
model.add(Dense(12, input_dim=300, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
ad = Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer=ad, metrics=['accuracy'])
history = model.fit(X, y, epochs=100, batch_size=64, validation_split=0.2)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
print(max(history.history['val_accuracy']))

## Speed up the learning

Even when we are using a slower learning rate, it is possible to speed up the training process. `BatchNormalization` is covered in the lectures and involves normalising all values going into a layer before the forward pass. Although it introduces some small extra computational cost into the forward pass, the results for speeding up learning rate can be spectacular - particularly in the deeper networs that we will look at in the next part of this exercise.

For now, import `BatchNormalization` from `tensorflow.keras.layers` and add a `BatchNormalization()` unit between each of your `Dense` layers.

Retrain the model and look at the learning rates.

In [ ]:
from tensorflow.keras.layers import BatchNormalization

In [ ]:
# define the keras model
from tensorflow.keras.optimizers import Adam
model = Sequential()
model.add(Dense(12, input_dim=300, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
ad = Adam(learning_rate=1e-4, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer=ad, metrics=['accuracy'])
history_bn = model.fit(X, y, epochs=100, batch_size=64, validation_split=0.2)

## Plot the results

Plot the same stats as for the previous training run.

## Look at some other metrics

Lets look at some other metrics of model performance

In [ ]:
y_true = y
y_pred = model.predict(X)

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

y_pred = np.rint(y_pred)
confusion_matrix(y_true, y_pred)

In [ ]:
plot_confusion_matrix(model, X, y,
                        display_labels=['No Ag', 'Ag'],
                        cmap=plt.cm.Blues,
                                 normalize=True)

In [ ]:
y_pred